In [1]:
# Importing libraries to be used later on in the script (cleaning, etc)
# the scripts stand alone to scrape data and output a csv, but these are imported for the scripts that clean the data

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [ ]:
# Loading another dataframe so I can import county FIPS codes and merge that on the zip code

# Simplemaps.com county & zip data
zip_cnty = pd.read_csv(os.path.join(path, 'Data', 'Original', 'GeoData', 'uszips-counties.csv'), dtype={'county_fips': str, 'zip' : str}, index_col = False)

# Realtor.com homes for sale scraper

In [1]:
# This should work any time going forward

import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import random
import csv
import re

start_time = time.time()

# Defining path
path=r'D:\Adam\Employment\Data Analysis Course\Final Data Project'

# Defining location of zipfile
zipfile = 'D:\Adam\Employment\Data Analysis Course\Final Data Project\Data\Cleaned\Tyson_zips_only.csv'

# Open the CSV file that contains the list of zip codes.
with open(zipfile, 'r') as f:
    reader = csv.reader(f)
    zip_codes = list(reader)

data = []    
    
# Loop through the list of zip codes.
for zip_code in zip_codes:
    print(f"Processing zip code: {zip_code[0]}")
    
    # Add a random delay before going to next zip code
    time.sleep(random.randint(30, 90))
    
    # Loop through the page numbers (1-20).
    for page_num in range(1, 20):
        print(f"Processing page {page_num}")
    
        # Update the URL in the script.
        url_template = "https://www.realtor.com/realestateandhomes-search/{}/pg-{}"
        url = url_template.format(zip_code[0], page_num)

        # Set user-agent header to avoid bot detection
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        # Add a random delay before making the request
        time.sleep(random.randint(8, 16))

        # Send HTTP GET request to the URL and get the HTML response
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("Request successful")
        else:
            print(f"Request failed with status code: {response.status_code}. No more listings pages found.")
            break

        # Parse the HTML response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the container for all the properties
        container = soup.find('section', class_='srp-content')

        # Check if the container is empty
        if container is None:
            print("No listings found")
            break

        # Check if the page contains the text "We're Sorry"
        if "We're Sorry" in soup.text:
            print("No more pages")
            break

        # Find all the listings in the container
        listings = container.find_all('div', attrs={'data-label': 'property-card'})

        # Loop through each listing and extract the details
        for listing in listings:
            if listing is not None:
                try:
                    address1 = listing.find('div', attrs={'data-label': 'pc-address'}).text.strip()
                except AttributeError:
                    address1 = ''

                try:
                    address2 = listing.find('div', attrs={'data-label': 'pc-address-second'}).text.strip()
                except AttributeError:
                    address2 = ''

                try:
                    availability = listing.find('span', class_='statusText').text.strip()
                except AttributeError:
                    style = ''

                try:
                    price = listing.find('span', attrs={'data-label': 'pc-price'}).text.strip().replace('From', '').replace(',', '').replace('$', '')
                except AttributeError:
                    price = ''

                try:
                    beds = listing.find('li', attrs={'data-label': 'pc-meta-beds'}).text.strip().replace('bed', '')
                except AttributeError:
                    beds = ''

                try:
                    baths = listing.find('li', attrs={'data-label': 'pc-meta-baths'}).text.strip().replace('bath', '')
                except AttributeError:
                    baths = ''

                try:
                    sqft = listing.find('li', attrs={'data-label': 'pc-meta-sqft'}).text.strip().replace(',', '').split('sqft')[0]
                except AttributeError:
                    sqft = ''

            else:
                print('No listing found.')

            # Appending the data
            data.append([str(zip_code[0]), address1, address2, availability, price, beds, baths, sqft])
       
        # stopping pagination if less than 41 listings found
        if soup.find(id="srp-footer-found-listing") is None:
            print("Not enough listings found for this zip code")
            break
        
        if int(soup.find(id="srp-footer-found-listing").text.strip().split()[1]) < 40:
            print("No more listings found")
            break
        
        else:
            continue
        
    print("Finished processing zip code")

# Create a Pandas dataframe from the extracted data
df = pd.DataFrame(data, columns=['TysonZip', 'Address1', 'Address2', 'Availability', 'Price', 'Beds', 'Baths', 'Sqft'])

# get today's date in the format YYYY-MM-DD
today = datetime.datetime.today().strftime('%Y-%m-%d')

# Export the dataframe to a CSV file
df.to_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_forsale_{today}.csv'), index=False)

# Print the dataframe
print(df)

end_time = time.time()

execution_time = end_time - start_time

print("Execution time:", execution_time)

Processing zip code: 73701
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 42602
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 35950
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 79108
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 44001
Processing page 1
Request successful
Processing page 2
Request successfu

Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 29640
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request successful
Processing page 5
Request successful
Processing page 6
Request successful
Processing page 7
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 66801
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 36027
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request successful
Processing page 5
R

Request successful
Processing page 8
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 35077
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 94580
Processing page 1
Request successful
No more listings found
Finished processing zip code
Processing zip code: 65301
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request successful
Processing page 5
Request successful
Processing page 6
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 78155
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request succes

### Checking and cleaning scraped for sale data

In [ ]:
# Importing scraped data

df_rs = pd.read_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_forsale_{today}.csv'), dtype={'TysonZip': str}, index_col = False)

In [ ]:
# checking dataset info

df_rs.info()

In [ ]:
# Looking for any other strange values in the Price column:

df_rs['Price'].value_counts()

In [ ]:
# Found a "Contact for Price" row and need to drop the whole row

df_rs.drop(df_rs[df_rs['Price'] == 'Contact For Price'].index, inplace=True)

In [ ]:
df_rs['Price'] = df_rs['Price'].astype('float64')

In [ ]:
# Checking for duplicates (and removing them if there are any) - then checking if the numbers change

df_rs.drop_duplicates(inplace=True)

df_rs.info()

In [ ]:
# Checking for missing values
missing_values = df_rs.isnull().sum()

# total rows in dataframe
total_rows = df_rs.shape[0]

# making it show me all the rows
pd.options.display.max_rows = len(df_rs.dtypes)

# calculating percentage of missing values
percent_missing = ((missing_values / total_rows) * 100).round(2)

# Making a dataframe to show missing values
missing_value_table = pd.concat([missing_values, percent_missing], axis=1)

# Naming columns for sorting
missing_value_table.columns = ['Missing Values', 'Percent Missing']

# sorting the view based on most missing values %
missing_value_table.sort_values('Percent Missing', ascending=True, inplace=True)

print(missing_value_table)

In [ ]:
# Sqft has a lot of missing values, and in looking at it, it appears that all the addresses with missing sqft values are
# just lots for sale. Which I don't believe helps the purpose of this analysis. So I'll remove all of those with missing
# sqft values. I'm not worried much about missing beds and baths info

df_rs.dropna(subset=['Sqft'], inplace=True)

In [ ]:
# Dropping all rows with missing addresses

df_rs.dropna(subset=['Address1'], inplace=True)

In [ ]:
# counting number of unique values in each column

print(df_rs.nunique())

In [ ]:
# Splitting up the Address2 data to create City, State, and Zip columns

df_rs['City'] = df_rs['Address2'].str.split(',', expand=True)[0]

df_rs['State'] = df_rs['Address2'].str.split(', ', expand=True)[1]
df_rs['State'] = df_rs['State'].str.split(' ', expand=True)[0]

# Some listings don't include zip code, so this will only capture those that do
df_rs['Zip'] = df_rs['Address2'].str.split(', ', expand=True)[1]
df_rs['Zip'] = df_rs['Zip'].str.split(' ', expand=True)[1]

In [ ]:
# dropping houses with duplicate addresses and sqft listed

df_rs.drop_duplicates(subset=['Address1', 'Sqft'], inplace=True)

In [ ]:
# Still counting unique values in each object column

df_rs['Address2'].value_counts()

In [ ]:
# Still counting unique values in each object column

df_rs['Availability'].value_counts()

In [ ]:
# Still counting unique values in each object column

# making it show me all the rows
pd.options.display.max_rows = len(df_rs['Beds'])

df_rs['Beds'].value_counts()

In [ ]:
# Converting all "Studio" listings to "1", as we'll assume it's roughly equivalent

df_rs['Beds'] = df_rs['Beds'].replace('Studio', '1')

In [ ]:
# converting Beds to float64 (I had to use float so I could conserve the NaN values)

df_rs['Beds'] = df_rs['Beds'].astype('float64')

In [ ]:
# Dropping rows with too many beds (i.e. apartment complexes or the like)

df_rs.drop(df_rs[df_rs['Beds'] > 14].index, inplace=True)

In [ ]:
# Still counting unique values in each object column

df_rs['Baths'].value_counts()

In [ ]:
# Replacing "+" values with normal values

df_rs['Baths'] = df_rs['Baths'].replace({'0.5+': '0.5', '1.5+': '1.5', '2.5+': '2.5', '3.5+': '3.5', '4.5+': '4.5', '5.5+': '5.5', '6.5+': '6.5', '7.5+': '7.5', '8.5+': '8.5', '9.5+': '9.5', '10.5+': '10.5', '11.5+': '11.5', '12.5+': '12.5'})

In [ ]:
# converting Baths to float64 (I had to use float so I could conserve the NaN values)

df_rs['Baths'] = df_rs['Baths'].astype('float64')

In [ ]:
# Still counting unique values in each object column

df_rs['Sqft'].value_counts(dropna=False)

In [ ]:
# dropping properties above 30,000 feet, as they're nearly always multi-family units

df_rs.drop(df_rs[df_rs['Sqft'] > 30000].index, inplace=True)

In [ ]:
# Still counting unique values in each object column

df_rs['City'].value_counts(dropna=False)

In [ ]:
# Still counting unique values in each object column

df_rs['State'].value_counts(dropna=False)

In [ ]:
# Still counting unique values in each object column

df_rs['Zip'].value_counts(dropna=False)

In [ ]:
# Identifying null values in Zip

# Set the display options to show all rows
pd.set_option('display.max_rows', None)

print(df_rs[df_rs['Zip'].isnull()])

In [ ]:
# Dropping null values in Zip (because they're all housing plan options for homes not yet built
# they're not specific homes at a specific address ready for purchase)

df_rs = df_rs.dropna(subset=['Zip'])

#### Merging county information to the dataframe

In [ ]:
# Adding the columns and values I want from the zip_cnty dataframe to my df_rs dataframe - useful for visualizations
df_rs = df_rs.merge(zip_cnty[['zip', 'lat', 'lng', 'county_fips', 'county_name']], 
                        left_on='Zip', right_on='zip', how='left')

# Remove the redundant 'zip' column from the merge result
df_rs.drop('zip', axis=1, inplace=True)

#showing results to confirm
df_rs.head()

In [ ]:
# Converting columns with leading zeroes to strings so that I can keep those leading zeroes when exporting

df_rs[['TysonZip', 'Zip', 'county_fips']] = df_rs[['TysonZip', 'Zip', 'county_fips']].astype(str)

In [2]:
# Exporting cleaned scraped dataset

df_rs.to_csv(os.path.join(path, 'Data', 'Cleaned', f'Realtor_Scraped_ForSale_{today}.csv'), index=False)

# Realtor.com Rental Listings Scraper

In [2]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import random
import csv
import re

start_time = time.time()

# Defining path
path=r'D:\Adam\Employment\Data Analysis Course\Final Data Project'

# Defining location of zipfile
zipfile = 'D:\Adam\Employment\Data Analysis Course\Final Data Project\Data\Cleaned\Tyson_zips_only.csv'

# Open the CSV file that contains the list of zip codes.
with open(zipfile, 'r') as f:
    reader = csv.reader(f)
    zip_codes = list(reader)

data = []    
    
# Loop through the list of zip codes.
for zip_code in zip_codes:
    print(f"Processing zip code: {zip_code[0]}")
    
    # Add a random delay before going to next zip code
    time.sleep(random.randint(30, 90))
    
    # Loop through the page numbers (1-20).
    for page_num in range(1, 20):
        print(f"Processing page {page_num}")
    
        # Update the URL in the script.
        url_template = "https://www.realtor.com/apartments/{}/pg-{}"
        url = url_template.format(zip_code[0], page_num)

        # Set user-agent header to avoid bot detection
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

        # Add a random delay before making the request
        time.sleep(random.randint(8, 16))

        # Send HTTP GET request to the URL and get the HTML response
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            print("Request successful")
        else:
            print(f"Request failed with status code: {response.status_code}. No more listings pages found.")
            break

        # Parse the HTML response using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the container for all the properties
        container = soup.find('section', class_=re.compile(r'PropertiesList_propertiesContainer'))

        # Check if the container is empty
        if container is None:
            print("No listings found")
            break

        # Check if the page contains the text "We're Sorry"
        if "We're Sorry" in soup.text:
            print("No more pages")
            break

        # Find all the listings in the container
        listings = container.find_all('div', class_=re.compile(r'BasePropertyCard_propertyCardWrap'))

        # Loop through each listing and extract the details
        for listing in listings:
            if listing is not None:
                try:
                    address1 = listing.find('div', {'class': 'truncate-line', 'data-testid': 'card-address-1'}).text.strip()
                except AttributeError:
                    address1 = ''

                try:
                    address2 = listing.find('div', {'class': 'truncate-line', 'data-testid': 'card-address-2'}).text.strip()
                except AttributeError:
                    address2 = ''

                try:
                    style = listing.find('div', {'class': 'message'}).text.split(' - ')[1]
                except AttributeError:
                    style = ''

                try:
                    price = listing.find('div', class_=re.compile(r'Price__Component')).text.strip().replace(',', '').replace('$', '')
                except AttributeError:
                    price = ''

                try:
                    beds = listing.find('li', {'data-testid': 'property-meta-beds'}).text.strip().replace('bed', '')
                except AttributeError:
                    beds = ''

                try:
                    baths = listing.find('li', {'data-testid': 'property-meta-baths'}).text.strip().replace('bath', '')
                except AttributeError:
                    baths = ''

                try:
                    sqft = listing.find('li', {'data-testid': 'property-meta-sqft'}).text.strip().replace(',', '').split('sqft')[0]
                except AttributeError:
                    sqft = ''

            else:
                print('No listing found.')

            # Split the beds, baths, and sqft values at the " - " and create two new rows with the values before and after the " - "

            if "-" in price:
                price_split = price.split(" - ")
                if "-" in beds:
                    beds_split = beds.split(" - ")
                    if "-" in sqft:
                        sqft_split = sqft.split(" - ")
                        if "-" in baths:
                            baths_split = baths.split(" - ")
                            data.append([str(zip_code[0]), address1, address2, style, price_split[0], beds_split[0], baths_split[0], sqft_split[0]])
                            data.append([str(zip_code[0]), address1, address2, style, price_split[1], beds_split[1], baths_split[1], sqft_split[1]])

                        else:
                            data.append([str(zip_code[0]), address1, address2, style, price_split[0], beds_split[0], baths, sqft_split[0]])
                            data.append([str(zip_code[0]), address1, address2, style, price_split[1], beds_split[1], baths, sqft_split[1]])
                    else:
                        data.append([str(zip_code[0]), address1, address2, style, price_split[0], beds_split[0], baths, sqft])
                        data.append([str(zip_code[0]), address1, address2, style, price_split[1], beds_split[1], baths, sqft])    
                else:
                    data.append([str(zip_code[0]), address1, address2, style, price_split[0], beds, baths, sqft])
                    data.append([str(zip_code[0]), address1, address2, style, price_split[1], beds, baths, sqft])
            else:
                data.append([str(zip_code[0]), address1, address2, style, price, beds, baths, sqft])
       
        # stopping pagination if less than 41 listings found
        if int(soup.find('div', class_=re.compile(r'MatchProperties')).text.strip().split(' matching')[0]) < 40:
            print("No more listings")
            break
        else:
            continue
        
    print("Finished processing zip code")

# Create a Pandas dataframe from the extracted data
df = pd.DataFrame(data, columns=['TysonZip', 'Address1', 'Address2', 'Style', 'Price', 'Beds', 'Baths', 'Sqft'])

# get today's date in the format YYYY-MM-DD
today = datetime.datetime.today().strftime('%Y-%m-%d')

# Export the dataframe to a CSV file
df.to_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_rentals_{today}.csv'), index=False)

# Print the dataframe
print(df)

end_time = time.time()

execution_time = end_time - start_time

print("Execution time:", execution_time)

Processing zip code: 73701
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 42602
Processing page 1
Request successful
No listings found
Finished processing zip code
Processing zip code: 35950
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 79108
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 44001
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 72764
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 72616
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 35031
Processing page 1
Request successful
No more listings
Finis

Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 76180
Processing page 1
Request successful
Processing page 2
Request successful
Processing page 3
Request successful
Processing page 4
Request successful
Processing page 5
Request successful
Processing page 6
Request failed with status code: 404. No more listings pages found.
Finished processing zip code
Processing zip code: 38261
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 68137
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 99363
Processing page 1
Request successful
No listings found
Finished processing zip code
Processing zip code: 50220
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 71602
Processing page 1
Request successful
No more listings
Finished processing zip code
Processing zip code: 04102
Processing p

### Checking and cleaning scraped rental data

In [ ]:
# Importing scraped data

df_rr = pd.read_csv(os.path.join(path, 'Data', 'Scraped', f'TysonZips_rentals_{today}.csv'), dtype={'TysonZip': str}, index_col = False)

In [ ]:
# checking dataset info

df_rr.info()

In [ ]:
# making it show me all the rows
pd.options.display.max_rows = len(df_rr['Price'].value_counts())

df_rr['Price'].value_counts()

In [ ]:
# Dropping rows with "Contact For Price" value

df_rr.drop(df_rr[df_rr['Price'] == 'Contact For Price'].index, inplace=True)

In [ ]:
# making Price an float64 dtype

df_rr = df_rr.astype({'Price': 'float64'})

In [ ]:
# making it show me all the rows
pd.options.display.max_rows = len(df_rr['Sqft'].value_counts())

df_rr['Sqft'].value_counts()

In [ ]:
# Keeping the lower sqft value from those that show sqft ranges, since these were properties that only listed a single
# price, and they were most likely listing "starting from" prices for a range of unit sizes (but I can't guess upper price)

df_rr['Sqft'] = df_rr['Sqft'].str.split(' - ').str[0]

In [ ]:
# Convert sqft to numeric while keeping null values as null

df_rr['Sqft'] = df_rr['Sqft'].astype('float64')

In [ ]:
# Checking for missing values

missing_values = df_rr.isnull().sum()

# total rows in dataframe
total_rows = df_rr.shape[0]

# making it show me all the rows
pd.options.display.max_rows = len(df_rr.dtypes)

# calculating percentage of missing values
percent_missing = ((missing_values / total_rows) * 100).round(2)

# Making a dataframe to show missing values
missing_value_table = pd.concat([missing_values, percent_missing], axis=1)

# Naming columns for sorting
missing_value_table.columns = ['Missing Values', 'Percent Missing']

# sorting the view based on most missing values %
missing_value_table.sort_values('Percent Missing', ascending=True, inplace=True)

print(missing_value_table)

If some of the unit details are missing (like sqft or beds, etc) that's not really a problem.  But if there's a lot of price data missing, that's a problem, because the goal here is to find out how many rental units are available and affordable (within a certain price range).

In [ ]:
# counting number of unique values in each column

print(df_rr.nunique())

This information is important primarily to determine how many Tyson zips (out of 95) have any rental units at all available for this scraping period

In [ ]:
# Checking value counts on each column to look for more cleaning to be done

df_rr['Style'].value_counts(dropna=False)

In [ ]:
# More value counts for each column

# making it show me all the rows
pd.set_option('display.max_rows', None)

df_rr['Beds'].value_counts(dropna=False)

In [ ]:
# I'm going to keep the first value in those that say "2-3" and stuff like that (for the same reason as above, because
# if they only listed one price, they probably listed the starting price, adn thus what would apply to the smallest unit)

df_rr['Beds'] = df_rr['Beds'].str.split(' - ').str[0]

In [ ]:
# Changing Studio+ to Studio
df_rr['Beds'] = df_rr['Beds'].replace({'Studio+': 'Studio'})

In [ ]:
# Changing value of "Studio" to 1 Bed so that Beds can be turned into numbers/float64

df_rr['Beds'] = df_rr['Beds'].replace({'Studio': '1'})

In [ ]:
df_rr['Beds'] = df_rr['Beds'].astype('float64')

In [ ]:
# More value counts for each column

df_rr['Baths'].value_counts(dropna=False)

In [ ]:
# Changing Baths+ to value without +

df_rr['Baths'] = df_rr['Baths'].replace({'5.5+': '5.5', '3.5+': '3.5', '2.5+': '2.5'})

In [ ]:
# Keeping the first number when there's a -, for the same reason as above

df_rr['Baths'] = df_rr['Baths'].str.split(' - ').str[0]

In [ ]:
# Changing Baths+ to value without +

df_rr['Baths'] = df_rr['Baths'].replace({'1.5+': '1.5', '2.5+': '2.5', '3.5+': '3.5', '4.5+': '4.5', '5.5+': '5.5'})

In [ ]:
df_rr['Baths'] = df_rr['Baths'].astype('float64')

In [ ]:
# Splitting Address 2 into city, state, and zip

df_rr['City'] = df_rr['Address2'].str.split(',', expand=True)[0]
# State abbreviation
df_rr['State'] = df_rr['Address2'].str.split(', ', expand=True)[1]
df_rr['State'] = df_rr['State'].str.split(' ', expand=True)[0]
# zip code
df_rr['Zip'] = df_rr['Address2'].str.split(', ', expand=True)[1]
df_rr['Zip'] = df_rr['Zip'].str.split(' ', expand=True)[1]

In [ ]:
# counting number of unique values in each column

print(df_rr.nunique())

It's important to compare Zip to TysonZip so that we can see how many additional areas found rental properties (within 3 mile radius of original search location) when no rental properties were found in the actual Tyson zip area

In [ ]:
df_rr.info()

In [ ]:
# More value counts for each column

df_rr['Address2'].value_counts(dropna=False)

In [ ]:
# More value counts for each column

df_rr['City'].value_counts(dropna=False)

In [ ]:
# More value counts for each column

df_rr['State'].value_counts(dropna=False)

In [ ]:
# More value counts for each column

df_rr['Zip'].value_counts(dropna=False)

In [ ]:
# Dropping all rows with missing addresses

df_rr.dropna(subset=['Address2'], inplace=True)

#### Merging county information to the dataframe

In [ ]:
# Adding the columns and values I want from the zip_cnty dataframe to my df_rr dataframe
df_rr = df_rr.merge(zip_cnty[['zip', 'lat', 'lng', 'county_fips', 'county_name']], 
                        left_on='Zip', right_on='zip', how='left')

# Remove the redundant 'zip' column from the merge result
df_rr.drop('zip', axis=1, inplace=True)

#showing results to confirm
df_rr.head()

In [ ]:
df_rr.info()

In [ ]:
# Converting columns with leading zeroes to strings so that I can keep those leading zeroes when exporting

df_rr[['TysonZip', 'Zip', 'county_fips']] = df_rs[['TysonZip', 'Zip', 'county_fips']].astype(str)

In [ ]:
# Exporting cleaned scraped dataset

df_rr.to_csv(os.path.join(path, 'Data', 'Cleaned', f'Realtor_Scraped_Rental_{today}.csv'), index=False)